In [1]:
# Cranfiel
import gdown
!gdown 1D48c_crSg6ia_OqZ5HoJmVmDLgn-iqq5
!gdown 1g0JYIiu-l6dSo5BpOuAeem2rVCkxNHjE

Downloading...
From: https://drive.google.com/uc?id=1D48c_crSg6ia_OqZ5HoJmVmDLgn-iqq5
To: /content/Cranfield.zip
100% 851k/851k [00:00<00:00, 135MB/s]
Downloading...
From: https://drive.google.com/uc?id=1g0JYIiu-l6dSo5BpOuAeem2rVCkxNHjE
To: /content/TEST.zip
100% 49.9k/49.9k [00:00<00:00, 122MB/s]


In [2]:
# Mô hình vector + Latent Semantic Index (LSI)

# Import cell

In [3]:
import nltk
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('punkt')

import string
from nltk.tokenize import word_tokenize, sent_tokenize
from nltk.stem.porter import PorterStemmer
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
import re
import numpy as np
import math

import string

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


# Cranfield - Reading data

In [4]:
!unzip 'Cranfield.zip'

Archive:  Cranfield.zip
   creating: Cranfield/
  inflating: Cranfield/1.txt         
  inflating: Cranfield/10.txt        
  inflating: Cranfield/100.txt       
  inflating: Cranfield/1000.txt      
  inflating: Cranfield/1001.txt      
  inflating: Cranfield/1002.txt      
  inflating: Cranfield/1003.txt      
  inflating: Cranfield/1004.txt      
  inflating: Cranfield/1005.txt      
  inflating: Cranfield/1006.txt      
  inflating: Cranfield/1007.txt      
  inflating: Cranfield/1008.txt      
  inflating: Cranfield/1009.txt      
  inflating: Cranfield/101.txt       
  inflating: Cranfield/1010.txt      
  inflating: Cranfield/1011.txt      
  inflating: Cranfield/1012.txt      
  inflating: Cranfield/1013.txt      
  inflating: Cranfield/1014.txt      
  inflating: Cranfield/1015.txt      
  inflating: Cranfield/1016.txt      
  inflating: Cranfield/1017.txt      
  inflating: Cranfield/1018.txt      
  inflating: Cranfield/1019.txt      
  inflating: Cranfield/102.txt       
  

In [5]:
!unzip 'TEST.zip'

Archive:  TEST.zip
  inflating: query.txt               
   creating: RES/
  inflating: RES/1.txt               
  inflating: RES/10.txt              
  inflating: RES/100.txt             
  inflating: RES/101.txt             
  inflating: RES/102.txt             
 extracting: RES/103.txt             
  inflating: RES/104.txt             
  inflating: RES/105.txt             
  inflating: RES/106.txt             
  inflating: RES/107.txt             
  inflating: RES/108.txt             
  inflating: RES/109.txt             
  inflating: RES/11.txt              
  inflating: RES/110.txt             
  inflating: RES/111.txt             
 extracting: RES/112.txt             
  inflating: RES/113.txt             
  inflating: RES/114.txt             
  inflating: RES/115.txt             
  inflating: RES/116.txt             
 extracting: RES/117.txt             
  inflating: RES/118.txt             
 extracting: RES/119.txt             
  inflating: RES/12.txt              
  inflating: 

In [6]:
# Reading the Cranfield's documents
import os

directory = '/content/Cranfield'  # Update with the path to your text files
file_extension = '.txt'  # Update with the file extension of your text files

text_files = [file for file in os.listdir(directory) if file.endswith(file_extension)]

CranfieldDocs = {}

for file_name in text_files:
    file_path = os.path.join(directory, file_name)
    with open(file_path, 'r') as file:
        contents = file.read()
        key = int(os.path.splitext(file_name)[0])  # Remove the file extension from the key
        CranfieldDocs[key] = contents

# Now you have the contents of each text file stored in the 'data_dict' dictionary

CranfieldDocs = dict(sorted(CranfieldDocs.items()))

In [7]:
# Loading the queries of Cranfield
def load_queries(file_path):
    queries = {}
    with open(file_path, 'r') as f:
        lines = f.readlines()
        for line in lines:
            key, value = line.strip().split('\t')
            queries[int(key)] = value
    return queries

CranfieldQueries = load_queries('/content/query.txt')
CranfieldQueries

{1: 'what similarity laws must be obeyed when constructing aeroelastic models of heated high speed aircraft .',
 2: 'what are the structural and aeroelastic problems associated with flight of high speed aircraft .',
 3: 'what problems of heat conduction in composite slabs have been solved so far .',
 4: 'can a criterion be developed to show empirically the validity of flow solutions for chemically reacting gas mixtures based on the simplifying assumption of instantaneous local chemical equilibrium .',
 5: 'what chemical kinetic system is applicable to hypersonic aerodynamic problems .',
 6: 'what theoretical and experimental guides do we have as to turbulent couette flow behaviour .',
 7: 'is it possible to relate the available pressure distributions for an ogive forebody at zero angle of attack to the lower surface pressures of an equivalent ogive forebody at angle of attack .',
 8: 'what methods -dash exact or approximate -dash are presently available for predicting body pressures at

In [8]:
## Loading the 'Gold' result
import os

folder_path = '/content/RES'
CranfieldTrue = {}

for file_name in os.listdir(folder_path):
    if file_name.endswith('.txt'):
        file_path = os.path.join(folder_path, file_name)
        with open(file_path, 'r') as file:
            lines = file.readlines()
            second_column = [int(line.split()[1]) for line in lines]
            key = int(file_name[:-4])  # Remove the '.txt' extension and convert to int
            CranfieldTrue[key] = second_column

CranfieldTrue = dict(sorted(CranfieldTrue.items()))

# Main Functions

## Porter Stemmer

In [9]:
def preprocessPorterStemmerOnly(docs):
    # docs is a dict {id: doc}
    porter_stemmer = PorterStemmer()
    terms = set()
    modified_docs = {}

    for i, doc in enumerate(docs):
        modified_doc = docs[doc].lower()
        modified_doc = re.sub(r'[^a-z0-9\s]', '', modified_doc)
        # use nltk to tokenize and filter the punctuation
        words = word_tokenize(modified_doc)
        sentence = ""
        for word in words:
            if word.isdigit() or word.isnumeric():
                continue
            if not word.isalnum():
                continue
            word = porter_stemmer.stem(word)
            terms.add(word)
            sentence += " " + word
        modified_docs[doc] = sentence

    return modified_docs, terms

In [10]:
# Porter Stemmer version of the Preprocess
def preprocessPorterStemmer(docs):
    # docs is a dict {id: doc}
    porter_stemmer = PorterStemmer()
    stop_words = set(stopwords.words('english'))
    terms = set()
    modified_docs = {}

    for i, doc in enumerate(docs):
        modified_doc = docs[doc].lower()
        modified_doc = re.sub(r'[^a-z0-9\s]', '', modified_doc)
        # use nltk to tokenize and filter the punctuation
        words = word_tokenize(modified_doc)
        sentence = ""
        for word in words:
            if word not in string.punctuation and word not in stop_words:
                if word.isdigit() or word.isnumeric():
                    continue
                if not word.isalnum():
                    continue
                word = porter_stemmer.stem(word)
                terms.add(word)
                sentence += " " + word
        modified_docs[doc] = sentence

    return modified_docs, terms

# docs, terms = preprocessPorterStemmer(CranfieldDocs)

## Lemmatizer

In [11]:
def preprocessLematizerOnly(docs):
    # docs is a dict {id: doc}
    lemmatizer = WordNetLemmatizer()
    terms = set()
    modified_docs = {}

    for i, doc in enumerate(docs):
        modified_doc = docs[doc].lower()
        modified_doc = re.sub(r'[^a-z0-9\s]', '', modified_doc)
        # use nltk to tokenize and filter the punctuation
        words = word_tokenize(modified_doc)
        sentence = ""
        for word in words:
            if word.isdigit() or word.isnumeric():
                continue
            if not word.isalnum():
                continue
            word = lemmatizer.lemmatize(word)
            terms.add(word)
            sentence += " " + word
        modified_docs[doc] = sentence

    return modified_docs, terms


In [12]:
def preprocessLematizer(docs):
    # docs is a dict {id: doc}
    lemmatizer = WordNetLemmatizer()
    stop_words = set(stopwords.words('english'))
    terms = set()
    modified_docs = {}

    for i, doc in enumerate(docs):
        modified_doc = docs[doc].lower()
        modified_doc = re.sub(r'[^a-z0-9\s]', '', modified_doc)
        # use nltk to tokenize and filter the punctuation
        words = word_tokenize(modified_doc)
        sentence = ""
        for word in words:
            if word not in string.punctuation and word not in stop_words:
                if word.isdigit() or word.isnumeric():
                    continue
                if not word.isalnum():
                    continue
                word = lemmatizer.lemmatize(word)
                terms.add(word)
                sentence += " " + word
        modified_docs[doc] = sentence

    return modified_docs, terms


## Indexing

In [13]:
def indexing(docs, terms):
    # docs is a dict which {id: doc}
    # terms is the set word not include punctuation
    # indexs is a dict {'word': set of doc_id}
    # order_term is a dict {'word': order in term}

    inverted_index = {}
    for doc_id, doc in docs.items():
        words = re.findall(r'\b\w+\b', doc)
        words = [word for word in words if word in terms]
        for word in words:
            if word in inverted_index:
                inverted_index[word].add(doc_id)
            else:
                inverted_index[word] = {doc_id}

    order_term = {}
    inverse_order_term = {}
    for i, word in enumerate(terms):
        order_term[word] = i
        inverse_order_term[i] = word
    return inverted_index, order_term, inverse_order_term
# indexs, order_term, inverse_order_term = indexing(docs, terms)

## Max ID

In [14]:
def find_max_id(docs):
    max_id = 0
    for doc in docs:
        max_id = max(max_id, doc)
    return max_id

# max_id = find_max_id(docs)
# print(f'The max of doc_id: {max_id}')


## Build Binary Matrix

In [15]:
def build_binary_matrix(docs, indexs, terms, order_term, inverse_order_term, max_id):
    # docs is a dict which {id: doc}
    # terms is the set word not include punctuation
    # indexs is a dict {'word': set of doc_id}
    # order_term is a dict {'word': order in term}
    # inverse_order_term is a dict {order in term: 'word'}

    num_docs  = max_id
    num_terms = len(terms)
    binary_matrix = np.zeros((num_terms, num_docs), dtype=int)
    for doc_id, doc in docs.items():
        for term in doc.split():
            if term in terms:
                term_index = order_term[term]
                binary_matrix[term_index, doc_id - 1] = 1
    return binary_matrix

## Build Frequency Matrix

In [16]:
def build_frequency_matrix(docs, indexs, terms, order_term, inverse_order_term, max_id):
    num_docs  = max_id
    num_terms = len(terms)
    frequency_matrix = np.zeros((num_terms, num_docs), dtype=int)
    # print(frequency_matrix)
    for doc_id, doc in docs.items():
        for term in doc.split():
            if term in terms:
                term_index = order_term[term]
                frequency_matrix[term_index, doc_id - 1] += 1
    return frequency_matrix

## Build TF-IDF Matrix

In [17]:
def build_tf_idf_matrix(docs, indexs, terms, order_term, inverse_order_term, max_id):
    num_docs    = max_id
    num_terms = len(terms)
    tf_idf_matrix  = np.zeros((num_terms, num_docs), dtype=float)
    frequency_matrix = build_frequency_matrix(docs, indexs, terms, order_term, inverse_order_term, max_id)
    for doc_id in range(1, num_docs + 1):
        for term_index in range(num_terms):
            term = inverse_order_term[term_index]
            if term in indexs:
                term_idf = len(indexs[term])
                tfik = frequency_matrix[term_index, doc_id - 1]
                idf = math.log10(num_docs / (1 + term_idf))
                tf_idf_matrix[term_index, doc_id - 1] = tfik * idf

    return tf_idf_matrix
# mt = build_tf_idf_matrix(docs, indexs, terms, order_term, inverse_order_term, max_id)

In [18]:

def build_tf_idf_matrixBest(docs, indexs, terms, order_term, inverse_order_term, max_id):
    num_docs = max_id
    num_terms = len(terms)
    tf_idf_matrix = np.zeros((num_terms, num_docs), dtype=float)
    binary_matrix = build_binary_matrix(docs, indexs, terms, order_term, inverse_order_term, max_id)

    for doc_id in range(1, num_docs + 1):
        for term_index in range(num_terms):
            term = inverse_order_term[term_index]
            if term in indexs:
                term_idf = len(indexs[term])
                tfik = binary_matrix[term_index, doc_id - 1]
                rel_docs = np.sum(binary_matrix[term_index])
                idf = np.log(num_docs / rel_docs)
                tf_idf_matrix[term_index, doc_id - 1] = tfik * idf

    for doc_id in range(num_docs):
        doc_vector = tf_idf_matrix[:, doc_id - 1]
        sum_w = np.sum(doc_vector ** 2)
        if sum_w != 0:
            tf_idf_matrix[:, doc_id - 1] /= sum_w

    return tf_idf_matrix
# build_tf_idf_matrix(docs, indexs, terms, order_term, inverse_order_term, max_id)

## Preprocess Query

In [19]:
def preprocess_query(query, indexs):
    # Preprocess the query and transform it into a numerical vector
    query_terms = query.split()
    preprocessed_query = np.zeros(len(indexs))
    for term in query_terms:
        if term in indexs:
            term_indices = indexs[term]
            for term_index in term_indices:
                preprocessed_query[term_index] += 1
    return preprocessed_query

In [20]:
def apply_stemming_or_lemmatization(sentence, remove_stopwords=False, use_stemmer=True, use_lemmatizer=False):
    if use_stemmer:
        stemmer = PorterStemmer()
    elif use_lemmatizer:
        lemmatizer = WordNetLemmatizer()

    stop_words = set(stopwords.words('english'))
    words = word_tokenize(sentence)

    if remove_stopwords:
        words = [word for word in words if word.lower() not in stop_words]

    if use_stemmer:
        stemmed_words = [stemmer.stem(word) for word in words]
        filtered_sentence = " ".join(stemmed_words)
    elif use_lemmatizer:
        lemmatized_words = [lemmatizer.lemmatize(word) for word in words]
        filtered_sentence = " ".join(lemmatized_words)
    else:
        filtered_sentence = " ".join(words)

    return filtered_sentence

## Calculate Similarity/LSI Model

In [21]:
## Base Code - No remove stopwords, no Stem - Base Code from Đạt

def calculate_similarity_NoProcess(docs, indexs, order_term, inverse_order_term, query, s, z, ut, n_top = 4):
    # docs is a dict which {id: doc}
    # indexs is a dict which {word: set of id_doc}
    # max_id is the bigger id
    # query is a sentence

    words = re.findall(r'\b\w+\b', query)
    words = [word for word in words if word in terms]

    query = ""
    for word in words:
        query = query + " " + word

    # max_id = find_max_id(docs)
    # frequency_matrix = build_binary_matrix(docs, indexs, terms, order_term, inverse_order_term, max_id)

   # calculate svd with the matrix
    # s, z, ut = np.linalg.svd(frequency_matrix, full_matrices=True)

    # truncate the s
    s = np.array(s)[:, :700]
    # truncate the z
    z = np.diag(z)[:700, :700]
    # truncate the ut
    ut = np.array(ut)[:700, :]

    k = s @ z
    d = z @ ut

    # calculate euclidean distance of d
    docs_ec = np.linalg.norm(d, axis=0)

    # get the vector of query
    order = [order_term[word] for word in words]
    query_arr = k[order]
    query_arr = np.sum(query_arr, axis=0)

    # caluculate euclidean distance of above
    query_ec = np.linalg.norm(query_arr)

    # calculate the cosine similarity
    results = query_arr @ d / (docs_ec * query_ec)
    # results = np.abs(results)
    # results = np.nan_to_num(results, nan=0)
    # results = np.nan_to_num(results, nan=-np.inf)
    return results

In [22]:
## Porter Stemming only - Keeping stopwords

def calculate_similarity_PorterStemmerOnly(docs, indexs, order_term, inverse_order_term, words, s, z, ut, n_top=4):
    # # Initialize PorterStemmer
    # porter_stemmer = PorterStemmer()

    # # Apply Porter stemming to the query words
    # words = re.findall(r'\b\w+\b', query)
    # words = [porter_stemmer.stem(word) for word in words if word in terms]

    query = ""
    for word in words:
        query = query + " " + word

    # truncate the s
    s = np.array(s)[:, :700]
    # truncate the z
    z = np.diag(z)[:700, :700]
    # truncate the ut
    ut = np.array(ut)[:700, :]
    k = s @ z
    d = z @ ut

    # calculate euclidean distance of d
    docs_ec = np.linalg.norm(d, axis=0)
    # get the vector of query
    order = [order_term[word] for word in words if word in order_term]
    query_arr = k[order]


    query_arr = np.sum(query_arr, axis=0)
    # caluculate euclidean distance of above
    query_ec = np.linalg.norm(query_arr)
    # calculate the cosine similarity
    results = query_arr @ d / (docs_ec * query_ec)
    # results = np.abs(results)
    return results

In [23]:
## Currently working code for Porter Stemmer and Remove Stopwords

def calculate_similarityPSRemoveStop(docs, indexs, order_term, inverse_order_term, query, s, z, ut, n_top=4):
    # docs is a dict which {id: doc}
    # indexs is a dict which {word: set of id_doc}
    # max_id is the bigger id
    # query is a sentence

    # Initialize Porter stemmer and stopwords
    stemmer = PorterStemmer()
    stop_words = set(stopwords.words('english'))

    # Preprocess the query
    words = re.findall(r'\b\w+\b', query)
    words = [word for word in words if word in terms]
    words = [stemmer.stem(word) for word in words if word not in stop_words]
    query = ""
    for word in words:
        query = query + " " + word

    # max_id = find_max_id(docs)
    # frequency_matrix = build_binary_matrix(docs, indexs, terms, order_term, inverse_order_term, max_id)

    # calculate svd with the matrix
    # s, z, ut = np.linalg.svd(frequency_matrix, full_matrices=True)

    # truncate the s
    s = np.array(s)[:, :700]
    # truncate the z
    z = np.diag(z)[:700, :700]
    # truncate the ut
    ut = np.array(ut)[:700, :]

    k = s @ z
    d = z @ ut

    # calculate euclidean distance of d
    docs_ec = np.linalg.norm(d, axis=0)

    # get the vector of query
    order = [order_term[word] for word in words if word in order_term]
    query_arr = k[order]
    query_arr = np.sum(query_arr, axis=0)

    # calculate euclidean distance of above
    query_ec = np.linalg.norm(query_arr)

    # calculate the cosine similarity
    results = query_arr @ d / (docs_ec * query_ec)
    # results = np.abs(results)
    # results = np.nan_to_num(results, nan=0)
    # results = np.nan_to_num(results, nan=-np.inf)
    return results

In [24]:
# Lemmatizer Only - Keeping Stopwords

def calculate_similarity_LemmatizeOnly(docs, indexs, order_term, inverse_order_term, query, s, z, ut, n_top=4):
    # Initialize WordNetLemmatizer
    lemmatizer = WordNetLemmatizer()

    # Lemmatize the query words
    words = re.findall(r'\b\w+\b', query)
    words = [lemmatizer.lemmatize(word) for word in words if word in terms]

    query = ""
    for word in words:
        query = query + " " + word

    # truncate the s
    s = np.array(s)[:, :700]
    # truncate the z
    z = np.diag(z)[:700, :700]
    # truncate the ut
    ut = np.array(ut)[:700, :]

    k = s @ z
    d = z @ ut

    # calculate euclidean distance of d
    docs_ec = np.linalg.norm(d, axis=0)

    # get the vector of query
    order = [order_term[word] for word in words if word in order_term]
    query_arr = k[order]
    query_arr = np.sum(query_arr, axis=0)

    # caluculate euclidean distance of above
    query_ec = np.linalg.norm(query_arr)

    # calculate the cosine similarity
    results = query_arr @ d / (docs_ec * query_ec)
    # results = np.abs(results)
    return results

In [25]:
# Lemmatizing and Removing Stopwords

def calculate_similarity_Lemmatize_Stopwords(docs, indexs, order_term, inverse_order_term, query, s, z, ut, n_top=4):
    # Initialize WordNetLemmatizer
    lemmatizer = WordNetLemmatizer()

    # Get the list of stopwords
    stop_words = set(stopwords.words('english'))

    # Lemmatize and remove stopwords from the query
    words = re.findall(r'\b\w+\b', query)
    words = [lemmatizer.lemmatize(word) for word in words if word not in stop_words and word in terms]

    query = ""
    for word in words:
        query = query + " " + word

    # truncate the s
    s = np.array(s)[:, :700]
    # truncate the z
    z = np.diag(z)[:700, :700]
    # truncate the ut
    ut = np.array(ut)[:700, :]

    k = s @ z
    d = z @ ut

    # calculate euclidean distance of d
    docs_ec = np.linalg.norm(d, axis=0)

    # get the vector of query
    order = [order_term[word] for word in words]
    query_arr = k[order]
    query_arr = np.sum(query_arr, axis=0)

    # caluculate euclidean distance of above
    query_ec = np.linalg.norm(query_arr)

    # calculate the cosine similarity
    results = query_arr @ d / (docs_ec * query_ec)
    # results = np.abs(results)
    return results

In [26]:
def calculate_similarityPSRemoveStopUpgrade(docs, indexs, order_term, inverse_order_term, query, s, z, ut, n_top=4):
    # Initialize Porter stemmer and stopwords
    stemmer = PorterStemmer()
    stop_words = set(stopwords.words('english'))

    # Precompile the regular expression pattern
    word_pattern = re.compile(r'\b\w+\b')

    # Preprocess the query
    words = word_pattern.findall(query)
    words = [stemmer.stem(word) for word in words if word in terms and word not in stop_words]
    query = " ".join(words)

    # max_id = find_max_id(docs)
    # frequency_matrix = build_binary_matrix(docs, indexs, terms, order_term, inverse_order_term, max_id)

    # Calculate SVD with the matrix
    # s, z, ut = np.linalg.svd(frequency_matrix, full_matrices=True)

    # Truncate the matrices
    s = s[:, :700]
    z = np.diag(z[:700])
    ut = ut[:700, :]

    k = s @ z
    d = z @ ut

    # Calculate Euclidean distance of d
    docs_ec = np.linalg.norm(d, axis=0)

    # Get the vector of query
    order = [order_term[word] for word in words if word in order_term]
    query_arr = k[order].sum(axis=0)

    # Calculate Euclidean distance of above
    query_ec = np.linalg.norm(query_arr)

    # Calculate the cosine similarity using matrix multiplication and element-wise operations
    results = (query_arr @ d) / (docs_ec * query_ec)
    # results = np.abs(results)

    return results

## Calculate MAP/AP of the query

In [27]:
def precision(y_true, y_pred):
  prec_list = []
  rec_list = []
  count = 0
  for i in range(len(y_pred)):
    if y_pred[i] in y_true:
      count += 1
      prec_list.append(count/(i+1))
      rec_list.append(count/len(y_true))
  return prec_list, rec_list
# list prec nội suy nè
def prec_in(prec_list, rec_list):
  prec_list_in = []
  for i in range(11):
    rec = i/10
    # prec_rec = max(rec->all)
    # lấy max từ prec_rec_in
    prec_rec_in = next((x[0] for x in enumerate(rec_list) if x[1] >= rec),-1)
    # print('rec_in:', prec_rec_in)
    prec_rec = 0
    if prec_rec_in != -1:
      prec_rec = max(prec_list[prec_rec_in:])
    prec_list_in.append(prec_rec)
  return prec_list_in

In [28]:
def MAP(list_y_true, list_y_pred, noisuy=True):
    common_keys = set(list_y_true.keys()).intersection(set(list_y_pred.keys()))
    result_dict = {}

    if noisuy:
        prec_in_data = {}
        for key in common_keys:
            prec_list, rec_list = precision(list_y_true[key], list_y_pred[key])
            prec_in_list = prec_in(prec_list, rec_list)
            # print('prec_list', prec_list)
            # print('rec_list', rec_list)
            # print('prec_list_in', prec_in_list)
            result_dict[key] = sum(prec_in_list) / len(prec_in_list)
            prec_in_data[key] = prec_in_list
        return prec_in_data, result_dict, sum(result_dict.values()) / len(result_dict.values())
    else:
        for key in common_keys:
            prec_list, rec_list = precision(list_y_true[key], list_y_pred[key])
            if len(prec_list) != 0:
                result_dict[key] = sum(prec_list) / len(prec_list)
            else:
                result_dict[key] = 0

    return result_dict, sum(result_dict.values()) / len(result_dict.values())

In [29]:
def count_P_R(list_y_true, list_y_pred):
  common_keys = set(list_y_true.keys()).intersection(set(list_y_pred.keys()))
  result_P_dict = {}
  result_R_dict = {}
  for key in common_keys:
      count = 0
      for i in range(len(list_y_pred[key])):
        if list_y_pred[key][i] in list_y_true[key]:
          count += 1
      if len(list_y_pred[key])!=0:
        result_P_dict[key] = count / len(list_y_pred[key])
      else:
        result_P_dict[key] = 0
      result_R_dict[key] = count / len(list_y_true[key])
  return result_P_dict, result_R_dict, sum(result_P_dict.values()) / len(result_P_dict.values()), sum(result_R_dict.values()) / len(result_R_dict.values())

In [30]:
def trec_11(prec_in_data):
  trec_11 = {}
  for i in range(11):
    trec_11[i/10] = 0
    for j in prec_in_data:
      trec_11[i/10]+=prec_in_data[j][i]/len(prec_in_data)
  return trec_11

# TF-IDF Best Combination from Vector Space (Binary, Inverse, Cosine)

## Running Cranfield Base - Keep stopwords, No Stemming

In [31]:
def preprocess(docs):
    # docs is a dict {id: doc}
    terms = set()
    modified_docs = {}

    for i, doc in enumerate(docs):
        modified_doc = docs[doc].lower()
        modified_doc = re.sub(r'[^a-z0-9\s]', '', modified_doc)
        # use nltk to tokenize and filter the punctuation
        words = word_tokenize(modified_doc)
        sentence = ""
        for word in words:
            if word not in string.punctuation:
                terms.add(word)
                sentence += " " + word
        modified_docs[doc] = sentence

    return modified_docs, terms


In [32]:
import time

start_time = time.time()
docs, terms = preprocess(CranfieldDocs)
indexs, order_term, inverse_order_term = indexing(docs, terms)

max_id = find_max_id(docs)

max_id = find_max_id(docs)
frequency_matrix = build_tf_idf_matrixBest(docs, indexs, terms, order_term, inverse_order_term, max_id)

# Calculate svd with the matrix
s, z, ut = np.linalg.svd(frequency_matrix, full_matrices=True)
total_timeBaseIndex_TFIDFBest = time.time() - start_time
print("Total execution time:", total_timeBaseIndex_TFIDFBest)

Total execution time: 187.1440191268921


In [33]:
print(f'number of word in terms: {len(terms)}')
print(f'number of word in indexs: {len(indexs)}')
print(f'number of document: {len(docs)}')
print(f'The max of doc_id: {max_id}')
print(f'The shape of the matrix: {frequency_matrix.shape}')

number of word in terms: 7908
number of word in indexs: 7908
number of document: 1400
The max of doc_id: 1400
The shape of the matrix: (7908, 1400)


In [34]:
from tqdm import tqdm

retrievalResult = dict()

start_time = time.time()

for key, value in tqdm(CranfieldQueries.items(), desc="Processing Queries", total=len(CranfieldQueries)):
    vector_similarity = calculate_similarity_NoProcess(docs, indexs, order_term, inverse_order_term, value, s, z, ut, 30)
    map_result = {}
    for result, doc_id in zip(vector_similarity, docs):
        map_result[result] = doc_id

    sorted_result = dict(sorted(map_result.items(), key=lambda x: x[0], reverse=True))
    top_results = list(sorted_result.values())

    retrievalResult[key] = top_results

total_timeBaseQuery_TFIDFBest = time.time() - start_time
print("Total execution time:", total_timeBaseQuery_TFIDFBest)

APTrecList_Base_TFIDFBest, APList_Base_TFIDFBest, MAP_Base_TFIDFBest = MAP(CranfieldTrue, retrievalResult)
APList_Base_TFIDFBest_NoInter, MAP_Base_TFIDFBest_NoInter = MAP(CranfieldTrue, retrievalResult, False)
l_p1, l_r1, p1, r1 = count_P_R(CranfieldTrue, retrievalResult)

print('\n')
print(MAP_Base_TFIDFBest)
print(MAP_Base_TFIDFBest_NoInter)
print(p1, r1)


Processing Queries: 100%|██████████| 225/225 [02:08<00:00,  1.75it/s]


Total execution time: 128.83027505874634


0.20757893167957592
0.19174589543595963
0.005831746031746031 1.0


## Running Cranfield Porter Stem + Remove Stopwords


In [35]:
start_time = time.time()

docs, terms = preprocessPorterStemmer(CranfieldDocs)
indexs, order_term, inverse_order_term = indexing(docs, terms)

max_id = find_max_id(docs)

max_id = find_max_id(docs)
frequency_matrix = build_tf_idf_matrixBest(docs, indexs, terms, order_term, inverse_order_term, max_id)

# Calculate svd with the matrix
s, z, ut = np.linalg.svd(frequency_matrix, full_matrices=True)

total_timePSStopIndex_TFIDFBest = time.time() - start_time
print("Total execution time:", total_timePSStopIndex_TFIDFBest)

Total execution time: 102.147784948349


In [36]:
print(f'number of word in terms: {len(terms)}')
print(f'number of word in indexs: {len(indexs)}')
print(f'number of document: {len(docs)}')
print(f'The max of doc_id: {max_id}')
print(f'The shape of the matrix: {frequency_matrix.shape}')


number of word in terms: 4556
number of word in indexs: 4556
number of document: 1400
The max of doc_id: 1400
The shape of the matrix: (4556, 1400)


In [37]:
from tqdm import tqdm

retrievalResult = dict()

start_time = time.time()

for key, value in tqdm(CranfieldQueries.items(), desc="Processing Queries", total=len(CranfieldQueries)):
    vector_similarity = calculate_similarityPSRemoveStop(docs, indexs, order_term, inverse_order_term, value, s, z, ut, 30)
    map_result = {}
    for result, doc_id in zip(vector_similarity, docs):
        map_result[result] = doc_id

    sorted_result = dict(sorted(map_result.items(), key=lambda x: x[0], reverse=True))
    top_results = list(sorted_result.values())

    retrievalResult[key] = top_results

total_timePSStopQuery_TFIDFBest = time.time() - start_time
print("Total execution time:", total_timePSStopQuery_TFIDFBest)

APTrecList_PorStemRemoveStop_TDIDFBest, APList_PorStemRemoveStop_TDIDFBest, MAP_PorStemRemoveStop_TDIDFBest = MAP(CranfieldTrue, retrievalResult)
APList_PorStemRemoveStop_TDIDFBest_NoInter, MAP_PorStemRemoveStop_TDIDFBest_NoInter = MAP(CranfieldTrue, retrievalResult, False)
l_p2, l_r2, p2, r2 = count_P_R(CranfieldTrue, retrievalResult)

print('\n')
print(MAP_PorStemRemoveStop_TDIDFBest)
print(MAP_PorStemRemoveStop_TDIDFBest_NoInter)
print(p2, r2)

Processing Queries:   6%|▌         | 14/225 [00:04<01:02,  3.40it/s]<ipython-input-23-be1da45f8717>:49: RuntimeWarning: invalid value encountered in true_divide
  results = query_arr @ d / (docs_ec * query_ec)
Processing Queries: 100%|██████████| 225/225 [01:13<00:00,  3.07it/s]


Total execution time: 73.28726172447205


0.1823429549152717
0.1675702672796301
0.005831746031746031 1.0


## Running Cranfield Lemmatizer + Remove Stopwords

In [38]:
start_time = time.time()

docs, terms = preprocessLematizer(CranfieldDocs)
indexs, order_term, inverse_order_term = indexing(docs, terms)

max_id = find_max_id(docs)
max_id = find_max_id(docs)
frequency_matrix = build_tf_idf_matrixBest(docs, indexs, terms, order_term, inverse_order_term, max_id)

# Calculate svd with the matrix
s, z, ut = np.linalg.svd(frequency_matrix, full_matrices=True)
total_timeLemmaStopIndex_TFIDFBest = time.time() - start_time
print("Total execution time:", total_timeLemmaStopIndex_TFIDFBest)


Total execution time: 145.02176690101624


In [39]:
print(f'number of word in terms: {len(terms)}')
print(f'number of word in indexs: {len(indexs)}')
print(f'number of document: {len(docs)}')
print(f'The max of doc_id: {max_id}')
print(f'The shape of the matrix: {frequency_matrix.shape}')


number of word in terms: 6424
number of word in indexs: 6424
number of document: 1400
The max of doc_id: 1400
The shape of the matrix: (6424, 1400)


In [40]:
retrievalResult = dict()

start_time = time.time()

for key, value in tqdm(CranfieldQueries.items(), desc="Processing Queries", total=len(CranfieldQueries)):
    vector_similarity = calculate_similarity_Lemmatize_Stopwords(docs, indexs, order_term, inverse_order_term, value, s, z, ut, 30)
    map_result = {}
    for result, doc_id in zip(vector_similarity, docs):
        map_result[result] = doc_id

    sorted_result = dict(sorted(map_result.items(), key=lambda x: x[0], reverse=True))
    top_results = list(sorted_result.values())

    retrievalResult[key] = top_results

total_timeLemmaStopQuery_TFIDFBest = time.time() - start_time
print("Total execution time:", total_timeLemmaStopQuery_TFIDFBest)

APTrecList_LemmaRemoveStop_TFIDFBest, APList_LemmaRemoveStop_TFIDFBest, MAP_LemmaRemoveStop_TFIDFBest = MAP(CranfieldTrue, retrievalResult)
APList_LemmaRemoveStop_TFIDFBest_NoInter, MAP_LemmaRemoveStop_TFIDFBest_NoInter = MAP(CranfieldTrue, retrievalResult, False)
l_p3, l_r3, p3, r3 = count_P_R(CranfieldTrue, retrievalResult)

print('\n')
print(MAP_LemmaRemoveStop_TFIDFBest)
print(MAP_LemmaRemoveStop_TFIDFBest_NoInter)
print(p3, r3)


Processing Queries: 100%|██████████| 225/225 [01:41<00:00,  2.23it/s]


Total execution time: 101.1265881061554


0.2688970648839272
0.2496992298884986
0.005831746031746031 1.0


## Running Cranfield Lemmatizer

In [41]:
start_time = time.time()

docs, terms = preprocessLematizerOnly(CranfieldDocs)
indexs, order_term, inverse_order_term = indexing(docs, terms)

max_id = find_max_id(docs)
frequency_matrix = build_tf_idf_matrixBest(docs, indexs, terms, order_term, inverse_order_term, max_id)

# Calculate svd with the matrix
s, z, ut = np.linalg.svd(frequency_matrix, full_matrices=True)
total_timeLemmaIndex_TFIDFBest = time.time() - start_time
print("Total execution time:", total_timeLemmaIndex_TFIDFBest)

Total execution time: 147.5000228881836


In [42]:
print(f'number of word in terms: {len(terms)}')
print(f'number of word in indexs: {len(indexs)}')
print(f'number of document: {len(docs)}')
print(f'The max of doc_id: {max_id}')
print(f'The shape of the matrix: {frequency_matrix.shape}')


number of word in terms: 6537
number of word in indexs: 6537
number of document: 1400
The max of doc_id: 1400
The shape of the matrix: (6537, 1400)


In [43]:

retrievalResult = dict()

start_time = time.time()

for key, value in tqdm(CranfieldQueries.items(), desc="Processing Queries", total=len(CranfieldQueries)):
    vector_similarity = calculate_similarity_LemmatizeOnly(docs, indexs, order_term, inverse_order_term, value, s, z, ut, 30)
    map_result = {}
    for result, doc_id in zip(vector_similarity, docs):
        map_result[result] = doc_id

    sorted_result = dict(sorted(map_result.items(), key=lambda x: x[0], reverse=True))
    top_results = list(sorted_result.values())

    retrievalResult[key] = top_results

total_timeLemmaQuery_TFIDFBest = time.time() - start_time
print("Total execution time:", total_timeLemmaQuery_TFIDFBest)

APTrecList_Lemma_TFIDFBest, APList_Lemma_TFIDFBest, MAP_Lemma_TFIDFBest = MAP(CranfieldTrue, retrievalResult)
APList_Lemma_TFIDFBest_NoInter, MAP_Lemma_TFIDFBest_NoInter = MAP(CranfieldTrue, retrievalResult, False)
l_p4, l_r4, p4, r4 = count_P_R(CranfieldTrue, retrievalResult)

print('\n')
print(MAP_Lemma_TFIDFBest)
print(MAP_Lemma_TFIDFBest_NoInter)
print(p4, r4)

Processing Queries: 100%|██████████| 225/225 [01:42<00:00,  2.19it/s]


Total execution time: 102.79377055168152


0.20654288658912856
0.19058881118332824
0.005831746031746031 1.0


## Running Cranfield PorterStemmer

In [44]:
start_time = time.time()

docs, terms = preprocessPorterStemmerOnly(CranfieldDocs)
indexs, order_term, inverse_order_term = indexing(docs, terms)

max_id = find_max_id(docs)
frequency_matrix = build_tf_idf_matrixBest(docs, indexs, terms, order_term, inverse_order_term, max_id)

# Calculate svd with the matrix
s, z, ut = np.linalg.svd(frequency_matrix, full_matrices=True)

total_timePSIndex_TFIDFBest = time.time() - start_time
print("Total execution time:", total_timePSIndex_TFIDFBest)

Total execution time: 106.55251049995422


In [45]:
print(f'number of word in terms: {len(terms)}')
print(f'number of word in indexs: {len(indexs)}')
print(f'number of document: {len(docs)}')
print(f'The max of doc_id: {max_id}')
print(f'The shape of the matrix: {frequency_matrix.shape}')


number of word in terms: 4665
number of word in indexs: 4665
number of document: 1400
The max of doc_id: 1400
The shape of the matrix: (4665, 1400)


In [46]:
retrievalResult = dict()

start_time = time.time()

for key, value in tqdm(CranfieldQueries.items(), desc="Processing Queries", total=len(CranfieldQueries)):
    vector_similarity = calculate_similarity_PorterStemmerOnly(docs, indexs, order_term, inverse_order_term, value, s, z, ut, 30)
    map_result = {}
    for result, doc_id in zip(vector_similarity, docs):
        map_result[result] = doc_id

    sorted_result = dict(sorted(map_result.items(), key=lambda x: x[0], reverse=True))
    top_results = list(sorted_result.values())

    retrievalResult[key] = top_results

total_timePSQuery_TFIDFBest = time.time() - start_time
print("Total execution time:", total_timePSQuery_TFIDFBest)

APTrecList_PorStem_TDIDFBest, APList_PorStem_TDIDFBest, MAP_PorStem_TDIDFBest = MAP(CranfieldTrue, retrievalResult)
APList_PorStem_TDIDFBest_NoInter, MAP_PorStem_TDIDFBest_NoInter = MAP(CranfieldTrue, retrievalResult, False)
l_p5, l_r5, p5, r5 = count_P_R(CranfieldTrue, retrievalResult)

print('\n')
print(MAP_PorStem_TDIDFBest)
print(MAP_PorStem_TDIDFBest_NoInter)
print(p5,r5)

Processing Queries: 100%|██████████| 225/225 [01:12<00:00,  3.10it/s]


Total execution time: 72.56257176399231


0.013378794534788407
0.011144080211372451
0.005831746031746031 1.0


## Thống kê

In [47]:
import pandas as pd
q1 = ["No Process", MAP_Base_TFIDFBest,MAP_Base_TFIDFBest_NoInter,p1,r1,total_timeBaseIndex_TFIDFBest, total_timeBaseQuery_TFIDFBest]
q2 = ["Stem + Stop", MAP_PorStemRemoveStop_TDIDFBest,MAP_PorStemRemoveStop_TDIDFBest_NoInter,p2,r2, total_timePSStopIndex_TFIDFBest, total_timePSStopQuery_TFIDFBest]
q3 = ["Lemma + Stop", MAP_LemmaRemoveStop_TFIDFBest,MAP_LemmaRemoveStop_TFIDFBest_NoInter,p3,r3, total_timeLemmaStopIndex_TFIDFBest, total_timeLemmaStopQuery_TFIDFBest]
q4 = ["Lemma", MAP_Lemma_TFIDFBest,MAP_Lemma_TFIDFBest_NoInter,p4,r4,total_timeLemmaIndex_TFIDFBest,total_timeLemmaQuery_TFIDFBest]
q5 = ["Stem", MAP_PorStem_TDIDFBest,MAP_PorStem_TDIDFBest_NoInter,p5,r5,total_timePSIndex_TFIDFBest,total_timePSQuery_TFIDFBest]

col = ["Type","Inter","NonInter","P","R","Index Time","Query Time"]
df1 = pd.DataFrame([q1,q2,q3,q4,q5], columns = col)

numeric_cols = ["Inter","NonInter","P","R","Index Time","Query Time"]
df1[numeric_cols] = df1[numeric_cols].round(4)

df1

,Type,Inter,NonInter,P,R,Index Time,Query Time
0,No Process,0.2076,0.1917,0.0058,1.0,187.1440,128.8303
1,Stem + Stop,0.1823,0.1676,0.0058,1.0,102.1478,73.2873
2,Lemma + Stop,0.2689,0.2497,0.0058,1.0,145.0218,101.1266
3,Lemma,0.2065,0.1906,0.0058,1.0,147.5000,102.7938
4,Stem,0.0134,0.0111,0.0058,1.0,106.5525,72.5626


# TF-IDF Base (Original Code)

## Cranfield Base

In [48]:
import time

In [49]:
start_time = time.time()
docs, terms = preprocess(CranfieldDocs)
indexs, order_term, inverse_order_term = indexing(docs, terms)

max_id = find_max_id(docs)
frequency_matrix = build_tf_idf_matrix(docs, indexs, terms, order_term, inverse_order_term, max_id)

# Calculate svd with the matrix
s, z, ut = np.linalg.svd(frequency_matrix, full_matrices=True)
total_timeBaseIndex_TFIDF = time.time() - start_time
print("Total execution time:", total_timeBaseIndex_TFIDF)

Total execution time: 84.44794082641602


In [50]:
print(f'number of word in terms: {len(terms)}')
print(f'number of word in indexs: {len(indexs)}')
print(f'number of document: {len(docs)}')
print(f'The max of doc_id: {(max_id)}')
print(f'The shape of the matrix: {frequency_matrix.shape}')


number of word in terms: 7908
number of word in indexs: 7908
number of document: 1400
The max of doc_id: 1400
The shape of the matrix: (7908, 1400)


In [51]:
from tqdm import tqdm

retrievalResult = dict()

start_time = time.time()

for key, value in tqdm(CranfieldQueries.items(), desc="Processing Queries", total=len(CranfieldQueries)):
    vector_similarity = calculate_similarity_NoProcess(docs, indexs, order_term, inverse_order_term, value, s, z, ut, 30)
    map_result = {}
    for result, doc_id in zip(vector_similarity, docs):
        map_result[result] = doc_id

    sorted_result = dict(sorted(map_result.items(), key=lambda x: x[0], reverse=True))
    top_results = list(sorted_result.values())

    retrievalResult[key] = top_results

total_timeBaseQuery_TFIDF = time.time() - start_time
print("Total execution time:", total_timeBaseQuery_TFIDF)

APTrecList_Base_TFIDFBase, APList_Base_TFIDFBase, MAP_Base_TFIDFBase = MAP(CranfieldTrue, retrievalResult)
APList_Base_TFIDFBase_NoInter, MAP_Base_TFIDFBase_NoInter = MAP(CranfieldTrue, retrievalResult, False)
l_p6, l_r6, p6, r6 = count_P_R(CranfieldTrue, retrievalResult)

print('\n')
print(MAP_Base_TFIDFBase)
print(MAP_Base_TFIDFBase_NoInter)
print(p6,r6)

Processing Queries: 100%|██████████| 225/225 [02:12<00:00,  1.69it/s]


Total execution time: 132.96679401397705


0.30315616584517474
0.282833465076131
0.005831746031746031 1.0


In [52]:
trec_11(APTrecList_Base_TFIDFBase)

{0.0: 0.6615246871075163,
 0.1: 0.6169137041924393,
 0.2: 0.4762446229554052,
 0.3: 0.3845372612509295,
 0.4: 0.3105360407277905,
 0.5: 0.2749768511944011,
 0.6: 0.2048166605643554,
 0.7: 0.15010426232444124,
 0.8: 0.11157487817900105,
 0.9: 0.07632540094057591,
 1.0: 0.06716345486006191}

## Cranfield Porter Stem Only

In [53]:
start_time = time.time()
docs, terms = preprocessPorterStemmerOnly(CranfieldDocs)
indexs, order_term, inverse_order_term = indexing(docs, terms)

max_id = find_max_id(docs)
frequency_matrix = build_tf_idf_matrix(docs, indexs, terms, order_term, inverse_order_term, max_id)

# Calculate svd with the matrix
s, z, ut = np.linalg.svd(frequency_matrix, full_matrices=True)
total_timePSIndex_TFIDF = time.time() - start_time
print("Total execution time:", total_timePSIndex_TFIDF)

Total execution time: 44.729166746139526


In [54]:
print(f'number of word in terms: {len(terms)}')
print(f'number of word in indexs: {len(indexs)}')
print(f'number of document: {len(docs)}')
print(f'The max of doc_id: {max_id}')
print(f'The shape of the matrix: {frequency_matrix.shape}')


number of word in terms: 4665
number of word in indexs: 4665
number of document: 1400
The max of doc_id: 1400
The shape of the matrix: (4665, 1400)


In [55]:
retrievalResult = dict()

start_time = time.time()

for key, value in tqdm(CranfieldQueries.items(), desc="Processing Queries", total=len(CranfieldQueries)):
    vector_similarity = calculate_similarity_PorterStemmerOnly(docs, indexs, order_term, inverse_order_term, value, s, z, ut, 30)
    map_result = {}
    for result, doc_id in zip(vector_similarity, docs):
        map_result[result] = doc_id

    sorted_result = dict(sorted(map_result.items(), key=lambda x: x[0], reverse=True))
    top_results = list(sorted_result.values())

    retrievalResult[key] = top_results

total_timePSQuery_TFIDF = time.time() - start_time
print("Total execution time:", total_timePSQuery_TFIDF)

APTrecList_PorStem_TFIDFBase, APList_PorStem_TFIDFBase, MAP_PorStem_TFIDFBase = MAP(CranfieldTrue, retrievalResult)
APList_PorStem_TFIDFBase_NoInter, MAP_PorStem_TFIDFBase_NoInter = MAP(CranfieldTrue, retrievalResult, False)
l_p7, l_r7, p7, r7 = count_P_R(CranfieldTrue, retrievalResult)

print('\n')
print(MAP_PorStem_TFIDFBase)
print(MAP_PorStem_TFIDFBase_NoInter)
print(p7,r7)

Processing Queries: 100%|██████████| 225/225 [01:14<00:00,  3.01it/s]


Total execution time: 74.87795090675354


0.016725275648737353
0.014557856815105519
0.005831746031746031 1.0


## Porter Stem + Remove Stopwords

In [56]:
start_time = time.time()
docs, terms = preprocessPorterStemmer(CranfieldDocs)
indexs, order_term, inverse_order_term = indexing(docs, terms)

max_id = find_max_id(docs)
frequency_matrix = build_tf_idf_matrix(docs, indexs, terms, order_term, inverse_order_term, max_id)

# Calculate svd with the matrix
s, z, ut = np.linalg.svd(frequency_matrix, full_matrices=True)
total_timePSStopIndex_TFIDF = time.time() - start_time
print("Total execution time:", total_timePSStopIndex_TFIDF)

Total execution time: 42.44214844703674


In [57]:
print(f'number of word in terms: {len(terms)}')
print(f'number of word in indexs: {len(indexs)}')
print(f'number of document: {len(docs)}')
print(f'The max of doc_id: {max_id}')
print(f'The shape of the matrix: {frequency_matrix.shape}')


number of word in terms: 4556
number of word in indexs: 4556
number of document: 1400
The max of doc_id: 1400
The shape of the matrix: (4556, 1400)


In [58]:

retrievalResult = dict()

start_time = time.time()

for key, value in tqdm(CranfieldQueries.items(), desc="Processing Queries", total=len(CranfieldQueries)):
    vector_similarity = calculate_similarityPSRemoveStop(docs, indexs, order_term, inverse_order_term, value, s, z, ut, 30)
    map_result = {}
    for result, doc_id in zip(vector_similarity, docs):
        map_result[result] = doc_id

    sorted_result = dict(sorted(map_result.items(), key=lambda x: x[0], reverse=True))
    top_results = list(sorted_result.values())

    retrievalResult[key] = top_results

total_timePSStopQuery_TFIDF = time.time() - start_time
print("Total execution time:", total_timePSStopQuery_TFIDF)

APTrecList_PorStemRemoveStop_TFIDFBase, APList_PorStemRemoveStop_TFIDFBase, MAP_PorStemRemoveStop_TFIDFBase = MAP(CranfieldTrue, retrievalResult)
APList_PorStemRemoveStop_TFIDFBase_NoInter, MAP_PorStemRemoveStop_TFIDFBase_NoInter = MAP(CranfieldTrue, retrievalResult, False)
l_p8, l_r8, p8, r8 = count_P_R(CranfieldTrue, retrievalResult)

print('\n')
print(MAP_PorStemRemoveStop_TFIDFBase)
print(MAP_PorStemRemoveStop_TFIDFBase_NoInter)
print(p8,r8)

Processing Queries:   6%|▌         | 14/225 [00:04<01:03,  3.33it/s]<ipython-input-23-be1da45f8717>:49: RuntimeWarning: invalid value encountered in true_divide
  results = query_arr @ d / (docs_ec * query_ec)
Processing Queries: 100%|██████████| 225/225 [01:13<00:00,  3.05it/s]


Total execution time: 73.72989702224731


0.2119482475804072
0.19598121769230983
0.005831746031746031 1.0


## Cranfield Lemma Only

In [59]:
start_time = time.time()
docs, terms = preprocessLematizerOnly(CranfieldDocs)
indexs, order_term, inverse_order_term = indexing(docs, terms)

max_id = find_max_id(docs)
frequency_matrix = build_tf_idf_matrix(docs, indexs, terms, order_term, inverse_order_term, max_id)

# Calculate svd with the matrix
s, z, ut = np.linalg.svd(frequency_matrix, full_matrices=True)
total_timeLemmaIndex_TFIDF = time.time() - start_time
print("Total execution time:", total_timeLemmaIndex_TFIDF)


Total execution time: 65.26884961128235


In [60]:
print(f'number of word in terms: {len(terms)}')
print(f'number of word in indexs: {len(indexs)}')
print(f'number of document: {len(docs)}')
print(f'The max of doc_id: {max_id}')
print(f'The shape of the matrix: {frequency_matrix.shape}')


number of word in terms: 6537
number of word in indexs: 6537
number of document: 1400
The max of doc_id: 1400
The shape of the matrix: (6537, 1400)


In [61]:

retrievalResult = dict()

start_time = time.time()

for key, value in tqdm(CranfieldQueries.items(), desc="Processing Queries", total=len(CranfieldQueries)):
    vector_similarity = calculate_similarity_LemmatizeOnly(docs, indexs, order_term, inverse_order_term, value, s, z, ut, 30)
    map_result = {}
    for result, doc_id in zip(vector_similarity, docs):
        map_result[result] = doc_id

    sorted_result = dict(sorted(map_result.items(), key=lambda x: x[0], reverse=True))
    top_results = list(sorted_result.values())

    retrievalResult[key] = top_results

total_timeLemmaQuery_TFIDF = time.time() - start_time
print("Total execution time:", total_timeLemmaQuery_TFIDF)

APTrecList_Lemma_TFIDFBase, APList_Lemma_TFIDFBase, MAP_Lemma_TFIDFBase = MAP(CranfieldTrue, retrievalResult)
APList_Lemma_TFIDFBase_NoInter, MAP_Lemma_TFIDFBase_NoInter = MAP(CranfieldTrue, retrievalResult, False)
l_p9, l_r9, p9, r9 = count_P_R(CranfieldTrue, retrievalResult)

print('\n')
print(MAP_Lemma_TFIDFBase)
print(MAP_Lemma_TFIDFBase_NoInter)
print(p9,r9)

Processing Queries: 100%|██████████| 225/225 [01:41<00:00,  2.22it/s]


Total execution time: 101.51218175888062


0.29175185937292675
0.27139216846906355
0.005831746031746031 1.0


## Lemma + Remove Stopwords

In [62]:
start_time = time.time()

docs, terms = preprocessLematizer(CranfieldDocs)
indexs, order_term, inverse_order_term = indexing(docs, terms)

max_id = find_max_id(docs)
frequency_matrix = build_tf_idf_matrix(docs, indexs, terms, order_term, inverse_order_term, max_id)

# Calculate svd with the matrix
s, z, ut = np.linalg.svd(frequency_matrix, full_matrices=True)

total_timeLemmaStopIndex_TFIDF = time.time() - start_time
print("Total execution time:", total_timeLemmaStopIndex_TFIDF)

Total execution time: 61.512091636657715


In [63]:
print(f'number of word in terms: {len(terms)}')
print(f'number of word in indexs: {len(indexs)}')
print(f'number of document: {len(docs)}')
print(f'The max of doc_id: {max_id}')
print(f'The shape of the matrix: {frequency_matrix.shape}')


number of word in terms: 6424
number of word in indexs: 6424
number of document: 1400
The max of doc_id: 1400
The shape of the matrix: (6424, 1400)


In [64]:

retrievalResult = dict()

start_time = time.time()

for key, value in tqdm(CranfieldQueries.items(), desc="Processing Queries", total=len(CranfieldQueries)):
    vector_similarity = calculate_similarity_Lemmatize_Stopwords(docs, indexs, order_term, inverse_order_term, value, s, z, ut, 30)
    map_result = {}
    for result, doc_id in zip(vector_similarity, docs):
        map_result[result] = doc_id

    sorted_result = dict(sorted(map_result.items(), key=lambda x: x[0], reverse=True))
    top_results = list(sorted_result.values())

    retrievalResult[key] = top_results

total_timeLemmaStopQuery_TFIDF = time.time() - start_time
print("Total execution time:", total_timeLemmaStopQuery_TFIDF)

APTrecList_LemmaRemoveStop_TFIDFBase, APList_LemmaRemoveStop_TFIDFBase, MAP_LemmaRemoveStop_TFIDFBase = MAP(CranfieldTrue, retrievalResult)
APList_LemmaRemoveStop_TFIDFBase_NoInter, MAP_LemmaRemoveStop_TFIDFBase_NoInter = MAP(CranfieldTrue, retrievalResult, False)
l_p10, l_r10, p10, r10 = count_P_R(CranfieldTrue, retrievalResult)

print('\n')
print(MAP_LemmaRemoveStop_TFIDFBase)
print(MAP_LemmaRemoveStop_TFIDFBase_NoInter)
print(p10,r10)

Processing Queries: 100%|██████████| 225/225 [01:44<00:00,  2.15it/s]


Total execution time: 104.82633590698242


0.33689905409603804
0.31799559389168386
0.005831746031746031 1.0


In [65]:
trec_11(APTrecList_LemmaRemoveStop_TFIDFBase)

{0.0: 0.6739856391711355,
 0.1: 0.6350849092071869,
 0.2: 0.5163042264794028,
 0.3: 0.42495697811081024,
 0.4: 0.3484077470466929,
 0.5: 0.3076785848722424,
 0.6: 0.23804853578894472,
 0.7: 0.18613240471686207,
 0.8: 0.15331564308870388,
 0.9: 0.11580518160510223,
 1.0: 0.106169744969333}

## Thống kê

In [66]:
import pandas as pd
q6 = ["No Process", MAP_Base_TFIDFBase,MAP_Base_TFIDFBase_NoInter,p6,r6, total_timeBaseIndex_TFIDF, total_timeBaseQuery_TFIDF]
q7 = ["PorterStem", MAP_PorStem_TFIDFBase,MAP_PorStem_TFIDFBase_NoInter,p7,r7,total_timePSIndex_TFIDF,total_timePSQuery_TFIDF]
q8 = ["Stem + Stop", MAP_PorStemRemoveStop_TFIDFBase,MAP_PorStemRemoveStop_TFIDFBase_NoInter,p8,r8,total_timePSStopIndex_TFIDF,total_timePSStopQuery_TFIDF]
q9 = ["Lemma", MAP_Lemma_TFIDFBase,MAP_Lemma_TFIDFBase_NoInter,p9,r9,total_timeLemmaIndex_TFIDF,total_timeLemmaQuery_TFIDF]
q10 = ["Lemma + Stop", MAP_LemmaRemoveStop_TFIDFBase,MAP_LemmaRemoveStop_TFIDFBase_NoInter,p10,r10,total_timeLemmaStopIndex_TFIDF,total_timeLemmaStopQuery_TFIDF]

col = ["Type","Inter","NonInter","P","R","Index Time","Query Time"]
df2 = pd.DataFrame([q6,q7,q8,q9,q10], columns = col)

numeric_cols = ["Inter","NonInter","P","R","Index Time","Query Time"]
df2[numeric_cols] = df2[numeric_cols].round(4)

df2

,Type,Inter,NonInter,P,R,Index Time,Query Time
0,No Process,0.3032,0.2828,0.0058,1.0,84.4479,132.9668
1,PorterStem,0.0167,0.0146,0.0058,1.0,44.7292,74.8780
2,Stem + Stop,0.2119,0.1960,0.0058,1.0,42.4421,73.7299
3,Lemma,0.2918,0.2714,0.0058,1.0,65.2688,101.5122
4,Lemma + Stop,0.3369,0.3180,0.0058,1.0,61.5121,104.8263


# Binary Matrix

## Cranfield Base

In [67]:
start_time = time.time()
docs, terms = preprocess(CranfieldDocs)
indexs, order_term, inverse_order_term = indexing(docs, terms)

max_id = find_max_id(docs)
frequency_matrix = build_binary_matrix(docs, indexs, terms, order_term, inverse_order_term, max_id)

# Calculate svd with the matrix
s, z, ut = np.linalg.svd(frequency_matrix, full_matrices=True)
total_timeBaseIndex_Binary = time.time() - start_time
print("Total execution time:", total_timeBaseIndex_Binary)

Total execution time: 29.8178231716156


In [68]:
print(f'number of word in terms: {len(terms)}')
print(f'number of word in indexs: {len(indexs)}')
print(f'number of document: {len(docs)}')
print(f'The max of doc_id: {max_id}')
print(f'The shape of the matrix: {frequency_matrix.shape}')


number of word in terms: 7908
number of word in indexs: 7908
number of document: 1400
The max of doc_id: 1400
The shape of the matrix: (7908, 1400)


In [69]:

from tqdm import tqdm

retrievalResult = dict()
start_time = time.time()
for key, value in tqdm(CranfieldQueries.items(), desc="Processing Queries", total=len(CranfieldQueries)):
    vector_similarity = calculate_similarity_NoProcess(docs, indexs, order_term, inverse_order_term, value, s, z, ut, 30)
    map_result = {}
    for result, doc_id in zip(vector_similarity, docs):
        map_result[result] = doc_id

    sorted_result = dict(sorted(map_result.items(), key=lambda x: x[0], reverse=True))
    top_results = list(sorted_result.values())

    retrievalResult[key] = top_results
total_timeBaseQuery_Binary = time.time() - start_time
print("Total execution time:", total_timeBaseQuery_Binary)

APTrecList_Base_Binary, APList_Base_Binary, MAP_Base_Binary= MAP(CranfieldTrue, retrievalResult)
APList_Base_Binary_NoInter, MAP_Base_Binary_NoInter = MAP(CranfieldTrue, retrievalResult, False)
l_p11, l_r11, p11, r11 = count_P_R(CranfieldTrue, retrievalResult)

print('\n')
print(MAP_Base_Binary)
print(MAP_Base_Binary_NoInter)
print(p11,r11)

Processing Queries: 100%|██████████| 225/225 [02:09<00:00,  1.73it/s]


Total execution time: 129.86451625823975


0.056619498250478094
0.0496762810429942
0.005831746031746031 1.0


## Cranfield Porter Stem Only

In [70]:
start_time = time.time()
docs, terms = preprocessPorterStemmerOnly(CranfieldDocs)
indexs, order_term, inverse_order_term = indexing(docs, terms)

max_id = find_max_id(docs)
frequency_matrix = build_binary_matrix(docs, indexs, terms, order_term, inverse_order_term, max_id)

# Calculate svd with the matrix
s, z, ut = np.linalg.svd(frequency_matrix, full_matrices=True)
total_timePSIndex_Binary = time.time() - start_time
print("Total execution time:", total_timePSIndex_Binary)

Total execution time: 16.382939100265503


In [71]:
print(f'number of word in terms: {len(terms)}')
print(f'number of word in indexs: {len(indexs)}')
print(f'number of document: {len(docs)}')
print(f'The max of doc_id: {max_id}')
print(f'The shape of the matrix: {frequency_matrix.shape}')


number of word in terms: 4665
number of word in indexs: 4665
number of document: 1400
The max of doc_id: 1400
The shape of the matrix: (4665, 1400)


In [72]:

retrievalResult = dict()
start_time = time.time()
for key, value in tqdm(CranfieldQueries.items(), desc="Processing Queries", total=len(CranfieldQueries)):
    vector_similarity = calculate_similarity_PorterStemmerOnly(docs, indexs, order_term, inverse_order_term, value, s, z, ut, 30)
    map_result = {}
    for result, doc_id in zip(vector_similarity, docs):
        map_result[result] = doc_id

    sorted_result = dict(sorted(map_result.items(), key=lambda x: x[0], reverse=True))
    top_results = list(sorted_result.values())

    retrievalResult[key] = top_results
total_timePSQuery_Binary = time.time() - start_time
print("Total execution time:", total_timePSQuery_Binary)
APTrecList_PorStem_Binary, APList_PorStem_Binary, MAP_PorStem_Binary = MAP(CranfieldTrue, retrievalResult)
APList_PorStem_Binary_NoInter, MAP_PorStem_Binary_NoInter = MAP(CranfieldTrue, retrievalResult, False)
l_p12, l_r12, p12, r12 = count_P_R(CranfieldTrue, retrievalResult)

print('\n')
print(MAP_PorStem_Binary)
print(MAP_PorStem_Binary_NoInter)
print(p12,r12)

Processing Queries: 100%|██████████| 225/225 [01:16<00:00,  2.92it/s]

Total execution time: 76.93225193023682


0.017038769184368252
0.01420289032774944
0.005831746031746031 1.0


## Porter Stem + Remove Stopwords

In [73]:
start_time = time.time()
docs, terms = preprocessPorterStemmer(CranfieldDocs)
indexs, order_term, inverse_order_term = indexing(docs, terms)

max_id = find_max_id(docs)
frequency_matrix = build_binary_matrix(docs, indexs, terms, order_term, inverse_order_term, max_id)

# Calculate svd with the matrix
s, z, ut = np.linalg.svd(frequency_matrix, full_matrices=True)
total_timePSStopIndex_Binary = time.time() - start_time
print("Total execution time:", total_timePSStopIndex_Binary)

Total execution time: 14.323012828826904


In [74]:
print(f'number of word in terms: {len(terms)}')
print(f'number of word in indexs: {len(indexs)}')
print(f'number of document: {len(docs)}')
print(f'The max of doc_id: {max_id}')
print(f'The shape of the matrix: {frequency_matrix.shape}')


number of word in terms: 4556
number of word in indexs: 4556
number of document: 1400
The max of doc_id: 1400
The shape of the matrix: (4556, 1400)


In [75]:

retrievalResult = dict()
start_time = time.time()
for key, value in tqdm(CranfieldQueries.items(), desc="Processing Queries", total=len(CranfieldQueries)):
    vector_similarity = calculate_similarityPSRemoveStop(docs, indexs, order_term, inverse_order_term, value, s, z, ut, 30)
    map_result = {}
    for result, doc_id in zip(vector_similarity, docs):
        map_result[result] = doc_id

    sorted_result = dict(sorted(map_result.items(), key=lambda x: x[0], reverse=True))
    top_results = list(sorted_result.values())

    retrievalResult[key] = top_results
total_timePSStopQuery_Binary = time.time() - start_time
print("Total execution time:", total_timePSStopQuery_Binary)
APTrecList_PorStemRemoveStop_Binary, APList_PorStemRemoveStop_Binary, MAP_PorStemRemoveStop_Binary = MAP(CranfieldTrue, retrievalResult)
APList_PorStemRemoveStop_Binary_NoInter, MAP_PorStemRemoveStop_Binary_NoInter = MAP(CranfieldTrue, retrievalResult, False)
l_p13, l_r13, p13, r13 = count_P_R(CranfieldTrue, retrievalResult)

print('\n')
print(MAP_PorStemRemoveStop_Binary)
print(MAP_PorStemRemoveStop_Binary_NoInter)
print(p13,r13)

Processing Queries:   6%|▌         | 14/225 [00:04<01:03,  3.32it/s]<ipython-input-23-be1da45f8717>:49: RuntimeWarning: invalid value encountered in true_divide
  results = query_arr @ d / (docs_ec * query_ec)
Processing Queries: 100%|██████████| 225/225 [01:14<00:00,  3.01it/s]


Total execution time: 74.6419084072113


0.1476416565841686
0.136305204771091
0.005831746031746031 1.0


## Cranfield Lemma Only

In [76]:
start_time = time.time()
docs, terms = preprocessLematizerOnly(CranfieldDocs)
indexs, order_term, inverse_order_term = indexing(docs, terms)

max_id = find_max_id(docs)
frequency_matrix = build_binary_matrix(docs, indexs, terms, order_term, inverse_order_term, max_id)

# Calculate svd with the matrix
s, z, ut = np.linalg.svd(frequency_matrix, full_matrices=True)
total_timeLemmaIndex_Binary = time.time() - start_time
print("Total execution time:", total_timeLemmaIndex_Binary)

Total execution time: 21.463600873947144


In [77]:
print(f'number of word in terms: {len(terms)}')
print(f'number of word in indexs: {len(indexs)}')
print(f'number of document: {len(docs)}')
print(f'The max of doc_id: {max_id}')
print(f'The shape of the matrix: {frequency_matrix.shape}')


number of word in terms: 6537
number of word in indexs: 6537
number of document: 1400
The max of doc_id: 1400
The shape of the matrix: (6537, 1400)


In [78]:

retrievalResult = dict()
start_time = time.time()
for key, value in tqdm(CranfieldQueries.items(), desc="Processing Queries", total=len(CranfieldQueries)):
    vector_similarity = calculate_similarity_LemmatizeOnly(docs, indexs, order_term, inverse_order_term, value, s, z, ut, 30)
    map_result = {}
    for result, doc_id in zip(vector_similarity, docs):
        map_result[result] = doc_id

    sorted_result = dict(sorted(map_result.items(), key=lambda x: x[0], reverse=True))
    top_results = list(sorted_result.values())

    retrievalResult[key] = top_results
total_timeLemmaQuery_Binary = time.time() - start_time
print("Total execution time:", total_timeLemmaQuery_Binary)
APTrecList_Lemma_Binary, APList_Lemma_Binary, MAP_Lemma_Binary = MAP(CranfieldTrue, retrievalResult)
APList_Lemma_Binary_NoInter, MAP_Lemma_Binary_NoInter = MAP(CranfieldTrue, retrievalResult, False)
l_p14, l_r14, p14, r14 = count_P_R(CranfieldTrue, retrievalResult)

print('\n')
print(MAP_Lemma_Binary)
print(MAP_Lemma_Binary_NoInter)
print(p14,r14)

Processing Queries: 100%|██████████| 225/225 [01:43<00:00,  2.17it/s]


Total execution time: 103.60214376449585


0.05306098592520308
0.04635314571160328
0.005831746031746031 1.0


## Lemma + Remove Stopwords

In [79]:
start_time = time.time()
docs, terms = preprocessLematizer(CranfieldDocs)
indexs, order_term, inverse_order_term = indexing(docs, terms)

max_id = find_max_id(docs)
frequency_matrix = build_binary_matrix(docs, indexs, terms, order_term, inverse_order_term, max_id)

# Calculate svd with the matrix
s, z, ut = np.linalg.svd(frequency_matrix, full_matrices=True)
total_timeLemmaStopIndex_Binary = time.time() - start_time
print("Total execution time:", total_timeLemmaStopIndex_Binary)

Total execution time: 22.992038011550903


In [80]:
print(f'number of word in terms: {len(terms)}')
print(f'number of word in indexs: {len(indexs)}')
print(f'number of document: {len(docs)}')
print(f'The max of doc_id: {max_id}')
print(f'The shape of the matrix: {frequency_matrix.shape}')


number of word in terms: 6424
number of word in indexs: 6424
number of document: 1400
The max of doc_id: 1400
The shape of the matrix: (6424, 1400)


In [81]:

retrievalResult = dict()
start_time = time.time()
for key, value in tqdm(CranfieldQueries.items(), desc="Processing Queries", total=len(CranfieldQueries)):
    vector_similarity = calculate_similarity_Lemmatize_Stopwords(docs, indexs, order_term, inverse_order_term, value, s, z, ut, 30)
    map_result = {}
    for result, doc_id in zip(vector_similarity, docs):
        map_result[result] = doc_id

    sorted_result = dict(sorted(map_result.items(), key=lambda x: x[0], reverse=True))
    top_results = list(sorted_result.values())

    retrievalResult[key] = top_results
total_timeLemmaStopQuery_Binary = time.time() - start_time
print("Total execution time:", total_timeLemmaStopQuery_Binary)
APTrecList_LemmaRemoveStop_Binary, APList_LemmaRemoveStop_Binary, MAP_LemmaRemoveStop_Binary = MAP(CranfieldTrue, retrievalResult)
APList_LemmaRemoveStop_Binary_NoInter, MAP_LemmaRemoveStop_Binary_NoInter = MAP(CranfieldTrue, retrievalResult, False)
l_p15, l_r15, p15, r15 = count_P_R(CranfieldTrue, retrievalResult)

print('\n')
print(MAP_LemmaRemoveStop_Binary)
print(MAP_LemmaRemoveStop_Binary_NoInter)
print(p15,r15)

Processing Queries: 100%|██████████| 225/225 [01:44<00:00,  2.15it/s]


Total execution time: 104.594961643219


0.21134334011936956
0.19328373400021456
0.005831746031746031 1.0


## Thống kê

In [82]:
import pandas as pd
q11 = ["No Process", MAP_Base_Binary,MAP_Base_Binary_NoInter,p11,r11,total_timeBaseIndex_Binary,total_timeBaseQuery_Binary]
q12 = ["PorterStem", MAP_PorStem_Binary,MAP_PorStem_Binary_NoInter,p12,r12,total_timePSIndex_Binary,total_timePSQuery_Binary]
q13 = ["Stem + Stop", MAP_PorStemRemoveStop_Binary,MAP_PorStemRemoveStop_Binary_NoInter,p13,r13,total_timePSStopIndex_Binary,total_timePSStopQuery_Binary]
q14 = ["Lemma", MAP_Lemma_Binary,MAP_Lemma_Binary_NoInter,p14,r14,total_timeLemmaIndex_Binary,total_timeLemmaQuery_Binary]
q15 = ["Lemma + Stop", MAP_LemmaRemoveStop_Binary,MAP_LemmaRemoveStop_Binary_NoInter,p15,r15,total_timeLemmaStopIndex_Binary,total_timeLemmaStopQuery_Binary]

col = ["Type","Inter","NonInter","P","R","Index Time","Query Time"]
df3 = pd.DataFrame([q11,q12,q13,q14,q15], columns = col)

numeric_cols = ["Inter","NonInter","P","R","Index Time","Query Time"]
df3[numeric_cols] = df3[numeric_cols].round(4)

df3

,Type,Inter,NonInter,P,R,Index Time,Query Time
0,No Process,0.0566,0.0497,0.0058,1.0,29.8178,129.8645
1,PorterStem,0.0170,0.0142,0.0058,1.0,16.3829,76.9323
2,Stem + Stop,0.1476,0.1363,0.0058,1.0,14.3230,74.6419
3,Lemma,0.0531,0.0464,0.0058,1.0,21.4636,103.6021
4,Lemma + Stop,0.2113,0.1933,0.0058,1.0,22.9920,104.5950


# Demo

In [83]:
print(docs[1])

 experimental investigation aerodynamics wing slipstream experimental study wing propeller slipstream made order determine spanwise distribution lift increase due slipstream different angle attack wing different free stream slipstream velocity ratio result intended part evaluation basis different theoretical treatment problem comparative span loading curve together supporting evidence showed substantial part lift increment produced slipstream due destalling boundary layer control effect integrated remaining lift increment subtracting destalling lift found agree well potential flow theory empirical evaluation destalling effect made specific configuration experiment


In [84]:
a = [[1,2],[3,4]]
b =[4,6]

In [85]:
# calculate_similarity(docs, indexs, max_id, "system is applicable")

In [86]:
indexs['system']

{32,
 39,
 40,
 42,
 62,
 77,
 78,
 83,
 84,
 92,
 103,
 130,
 151,
 152,
 163,
 172,
 199,
 202,
 216,
 229,
 274,
 290,
 332,
 342,
 344,
 349,
 367,
 368,
 395,
 396,
 416,
 448,
 451,
 472,
 506,
 511,
 515,
 529,
 623,
 624,
 642,
 650,
 724,
 790,
 793,
 800,
 813,
 824,
 828,
 847,
 848,
 849,
 873,
 894,
 914,
 930,
 943,
 952,
 958,
 961,
 968,
 1032,
 1035,
 1047,
 1065,
 1077,
 1088,
 1102,
 1119,
 1147,
 1169,
 1186,
 1200,
 1226,
 1254,
 1265,
 1271,
 1277,
 1280,
 1283,
 1298,
 1316,
 1336,
 1389}

In [87]:
matrix = [[1,0,1,0,0],
            [1,1,0,0,0],
            [0,1,0,0,0],
            [0,1,1,0,0],
            [0,0,1,1,0],
            [0,0,0,1,0],
            [0,0,0,1,0],
            [0,0,0,1,1]]

In [88]:
s, z, ut = np.linalg.svd(matrix, full_matrices=True)

In [89]:
s = np.array(s)
s = s[:, :2]
print(s)

[[-0.39615277  0.28005737]
 [-0.31426806  0.44953214]
 [-0.17823952  0.26899154]
 [-0.43836375  0.36850831]
 [-0.52400482 -0.24640466]
 [-0.26388058 -0.34592143]
 [-0.26388058 -0.34592143]
 [-0.32637322 -0.45966878]]


In [90]:
ut = np.array(ut)
ut = ut[:2, :]
print(ut)

[[-0.31086574 -0.40733041 -0.59446137 -0.60304575 -0.1428143 ]
 [ 0.36293322  0.54074246  0.20005441 -0.6953914  -0.22866156]]


In [91]:
z = np.diag(z)
z = z[:2, :2]
print(z)

[[2.28529793 0.        ]
 [0.         2.01025824]]


In [92]:
k = s @ z
print(k)

[[-0.90532712  0.56298763]
 [-0.71819615  0.90367568]
 [-0.40733041  0.54074246]
 [-1.00179178  0.74079687]
 [-1.19750713 -0.49533699]
 [-0.60304575 -0.6953914 ]
 [-0.60304575 -0.6953914 ]
 [-0.74586005 -0.92405295]]


In [93]:
d = z @ ut
print(d)

[[-0.71042084 -0.93087134 -1.35852135 -1.37813921 -0.32637322]
 [ 0.7295895   1.08703198  0.40216102 -1.39791629 -0.45966878]]


In [94]:
k[3] + k[4]

array([-2.19929891,  0.24545988])

In [95]:
query_arr = k[3] + k[4]
query_ec = np.linalg.norm(query_arr)
print(query_ec)

2.2129541887444324


In [96]:
docs_ec = np.linalg.norm(d, axis=0)
print(docs_ec)

[1.01833129 1.4311394  1.416797   1.96301748 0.56375072]


In [97]:
results = query_arr @ d / (docs_ec * query_ec)

In [98]:
map_reuslt = {}
for result, doc_id in zip(results, docs):
    map_reuslt[result] = doc_id
    # print(result)

In [99]:
sorted_result = dict(sorted(map_reuslt.items(), key=lambda x: x[0], reverse=True))

In [100]:
n_top = 4
for i, result in enumerate(sorted_result):
    print(sorted_result[result])
    if i == n_top - 1:
        break

3
1
2
4
